In [2]:
# Import dependencies
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
from collections import Counter
import psycopg2
from psycopg2 import Error
from config import DB_PASS

In [3]:
# Connection parameters, yours will be different
param_dic = {
    "host"      : "ec2-34-239-241-121.compute-1.amazonaws.com",
    "database"  : "dfm419pf436p66",
    "user"      : "eulpledavzofae",
    "password"  : DB_PASS
}
def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn

In [4]:
def postgresql_to_dataframe(conn, select_query, column_names):
    """
    Tranform a SELECT query into a pandas dataframe
    """
    cursor = conn.cursor()
    try:
        cursor.execute(select_query)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        cursor.close()
        return 1
    
    #list of tupples
    tupples = cursor.fetchall()
    cursor.close()
    
    # turn it into a pandas dataframe
    heart_dp = pd.DataFrame(tupples, columns=column_names)
    return heart_dp

In [7]:
# Connect to the database
conn = connect(param_dic)
column_names = ["Pt_ID", "Age", "Sex", "ChestPainType", "RestingBP", "Cholesterol", "FastingBS","RestingEKG","MaxHR","ExerciseAngina","OldPeak","ST_Slope", "HeartDisease"]
# Execute the "SELECT *" query
heart_disease_df = postgresql_to_dataframe(conn, "select * from heart_dp_train", column_names)
heart_disease_df.head()

Connecting to the PostgreSQL database...
Connection successful


,Pt_ID,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingEKG,MaxHR,ExerciseAngina,OldPeak,ST_Slope,HeartDisease
0,25047,74,M,NAP,138,0,0,Normal,116,N,0.2,Up,Absence
1,25177,60,M,ASY,130,186,1,ST,140,Y,0.5,Flat,Presence
2,25498,40,M,ASY,125,0,1,Normal,165,N,0,Flat,Presence
3,25189,69,M,ASY,140,208,0,ST,140,Y,2,Flat,Presence
4,25434,62,M,ASY,158,210,1,Normal,112,Y,3,Down,Presence


In [8]:
# Confirm entry count
heart_disease_df['HeartDisease'].value_counts()

Presence    508
Absence     410
Name: HeartDisease, dtype: int64

In [12]:
# Encode object variables with Scikit Learn
object_columns = heart_disease_df.dtypes[heart_disease_df.dtypes == "object"].index.tolist()
le = LabelEncoder()
encoded_df = heart_disease_df.copy()

for column in object_columns:
    encoded_df[column] = le.fit_transform(encoded_df[column])
    
encoded_df.head()

,Pt_ID,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingEKG,MaxHR,ExerciseAngina,OldPeak,ST_Slope,HeartDisease
0,25047,74,1,2,138,0,0,1,116,0,12,2,0
1,25177,60,1,0,130,186,1,2,140,1,15,1,1
2,25498,40,1,0,125,0,1,1,165,0,10,1,1
3,25189,69,1,0,140,208,0,2,140,1,30,1,1
4,25434,62,1,0,158,210,1,1,112,1,39,0,1


In [13]:
# Define the target and features
y = encoded_df['HeartDisease'].ravel()

X = encoded_df.drop('HeartDisease', axis = 1)

In [14]:
# Split in to test and train sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 22)